<a href="https://colab.research.google.com/github/jasonheesanglee/LLM_Study/blob/main/RAG_speaker_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!nvidia-smi

Wed Feb 14 05:21:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git
!pip install -qqq torch python-multipart cohere kaleido openai tiktoken sentence_transformers accelerate transformers[torch] chromadb langchain
!pip install -qqq bitsandbytes huggingface
!wget -qqq https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1/resolve/main/pytorch_model.bin -O pytorch_model.bin
!wget -qqq https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1/resolve/main/config.json -O config.json

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [13]:
!pip show accelerate

Name: accelerate
Version: 0.28.0.dev0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft


In [14]:
from google.colab import drive
drive.mount('/content/drive')

import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OpenAI_API')
os.environ['HF_AUTH_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['GOOGLE_API_KEY'] = userdata.get('GoogleDevAPI')

from accelerate import Accelerator, notebook_launcher
from accelerate.utils import set_seed

Mounted at /content/drive


In [15]:
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [16]:
accelerator = Accelerator()
device = accelerator.device

In [17]:
model_id = 'Open-Orca/OpenOrca-Platypus2-13B'
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_id,
    config=AutoConfig.from_pretrained(model_id),
    quantization_config=bnb_config,
    device_map='cuda',
)

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

In [18]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32002, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

In [19]:
# messages = [
#     {'role':'user', 'content':'Please list up the characters in the wonderful wizard of oz'}
# ]

# encodeds = tokenizer.apply_chat_template(messages, return_tensors='pt')

# model_input = encodeds.to(device)

In [20]:
# generated_ids = model.generate(inputs=model_input, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [21]:
import gc
# gc.collect()
# torch.cuda.empty_cache()

In [22]:
import locale
def getpreferredencoding(do_setlocale=True):
    return 'UTF-8'

locale.getpreferredencoding = getpreferredencoding

In [23]:
!pip -qqq install langchain pypdf chromadb sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.0 MB/s eta 0:00:00


In [24]:
import zipfile

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/dreaming/emotional_reading/gutenberg-over-70000.zip')
zip_ref.extract(member='1-500/55_TheWonderfulWizard.pkl', path='/content/')

'/content/1-500/55_TheWonderfulWizard.pkl'

In [25]:
import pickle
with open('/content/1-500/55_TheWonderfulWizard.pkl', 'rb') as f:
    data = pickle.load(f)
print(data.split()[:5])
with open('/content/sample_text.txt', 'w', encoding='utf-8') as f:
    f.write(data)

with open('/content/sample_text.txt', encoding='utf-8') as f:
    sample = f.read()

['\ufeffThe', 'Project', 'Gutenberg', 'eBook', 'of']


In [26]:
import tiktoken
tiktokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
    tokens = tiktokenizer.encode(text)
    return len(tokens)

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=tiktoken_len,
)
gc.collect()
torch.cuda.empty_cache()

In [28]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

texts = text_splitter.split_text(sample)
print(texts[0])

gc.collect()
torch.cuda.empty_cache()

﻿The Project Gutenberg eBook of The Wonderful Wizard of Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Wonderful Wizard of Oz


Author: L. Frank Baum

Release date: February 1, 1993 [eBook #55]
                Most recently updated: March 30, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***
[Illustration]




The Wonderful Wizard of Oz

by L. Frank Baum


This book is dedicated to my good friend & comrade
My Wife
L.F.B.


Contents

 Introduction
 Chapter I. The Cyclone
 Chapter II. The Council with the Munchkins
 Chapter III. Ho

In [29]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=1000,
)

In [30]:
prompt_template = '''
### [INST]
Instruction: You are a professional novel reader. Please answer the questions after a careful review of the provided document.
Here is context to help:

{context}

### Question
{question}
[/INST]
'''

llm_orca_platypus = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)
llm_chain = LLMChain(llm=llm_orca_platypus, prompt=prompt)
gc.collect()
torch.cuda.empty_cache()

In [31]:
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'all-MiniLM-L6-v2'
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)
gc.collect()
torch.cuda.empty_cache()
# db = FAISS.from_texts(texts, hf)
# retriever = db.as_retriever(
#     search_type='similarity',
#     search_kwargs={'k':3}
# )
# gc.collect()
# torch.cuda.empty_cache()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
# db.save_local('/content/drive/MyDrive/Colab Notebooks/dreaming/emotional_reading/amazing_wizard_of_oz')

In [33]:
# db = FAISS.from_texts(texts, hf)
new_db = FAISS.load_local('/content/drive/MyDrive/Colab Notebooks/dreaming/emotional_reading/amazing_wizard_of_oz', hf)
retriever = new_db.as_retriever(
    search_type='mmr',
    search_kwargs={'k':3}
)
rag_chain = (
    {'context':retriever, 'question':RunnablePassthrough()}
    | llm_chain
)
gc.collect()
torch.cuda.empty_cache()


In [34]:
result = rag_chain.invoke('''
Caution : Please only return the result of "Request 1".
Request 1. Please list up all the main and minor characters.
''')

In [35]:
print(f"Answer : {result['text']}")
print(f'Question : {result["question"]}')
print(f'Context : {result["context"]}')
gc.collect()
torch.cuda.empty_cache()


Answer : 
Here is the list of main and minor characters in the provided document:

Main Characters:
1. Dorothy
2. Tin Woodman
3. Scarecrow
4. Cowardly Lion
5. Queen of the Mice

Minor Characters:
1. Toto
2. Wildcat
3. Mice
4. Tin Man
5. Lion
6. Scarecrow
7. Wicked Witch of the East
8. Wizard
9. Green Girl
10. Soldier

Please note that this list is not exhaustive, as it only includes the characters mentioned in the provided text. There might be other characters not mentioned in the text.
Question : 
Caution : Please only return the result of "Request 1".
Request 1. Please list up all the main and minor characters.

Context : [Document(page_content='At that moment several mice were seen running up as fast as their\nlittle legs could carry them, and when they saw their Queen they\nexclaimed:\n\n“Oh, your Majesty, we thought you would be killed! How did you manage\nto escape the great Wildcat?” They all bowed so low to the little Queen\nthat they almost stood upon their heads.\n\n“This fun

In [36]:
main_char = result['text'].split('\n\n')[1]
minor_char = result['text'].split('\n\n')[2]

characters = []
for idx, char in enumerate(main_char.split('. ')):
    if idx ==0:
        continue
    else:
        characters.append(char.split('\n')[0])
for idx, char in enumerate(minor_char.split('. ')):
    if idx ==0:
        continue
    else:
        characters.append(char.split('\n')[0])
characters

['Dorothy',
 'Tin Woodman',
 'Scarecrow',
 'Cowardly Lion',
 'Queen of the Mice',
 'Toto',
 'Wildcat',
 'Mice',
 'Tin Man',
 'Lion',
 'Scarecrow',
 'Wicked Witch of the East',
 'Wizard',
 'Green Girl',
 'Soldier']

In [37]:
result = rag_chain.invoke(f"""
Request 1. Please elaborate the personality or characteristics of each characters in the following list.
{', '.join(characters)}
""")

In [38]:
print(f"Answer : {result['text']}")
print(f'Question : {result["question"]}')
print(f'Context : {result["context"]}')
gc.collect()
torch.cuda.empty_cache()


Answer : 
Here's a brief description of each character's personality or characteristics:

1. Dorothy: A young girl from Kansas, who finds herself in the magical land of Oz. She is brave, resourceful, and caring, as she embarks on a journey to find her way back home.

2. Tin Woodman: A former woodcutter who was transformed into a tin man by the Wicked Witch of the West. He is kind, compassionate, and longs for a heart.

3. Scarecrow: A farmhand who was given a brain by the Wizard of Oz. He is intelligent, wise, and often serves as a leader among his friends.

4. Cowardly Lion: A former king of the forest who was cursed by the Wicked Witch of the West. He is brave, loyal, and seeks courage.

5. Queen of the Mice: A small, kind, and wise ruler of the Munchkin Country. She is a loyal friend to Dorothy and her companions.

6. Toto: Dorothy's small dog, who accompanies her on her journey through Oz. He is brave, loyal, and often helps his companions in their adventures.

7. Wildcat: A creatu

In [39]:
result = rag_chain.invoke(f"""
Request 1. Please list up the nicknames of each characters in the list below refered in text.
Caution: Do not consider any word with 's as a name of the character, such as, Cowardly Lion's courage, Scarecrow's head, etc.
{', '.join(characters)}

Example 1. Dorothy : Little Girl, etc.
Example 2. Toto : Little black dog, etc.
""")

print(f"Answer : {result['text']}")
print(f'Question : {result["question"]}')
print(f'Context : {result["context"]}')
gc.collect()
torch.cuda.empty_cache()


Answer : 
Nicknames:
1. Dorothy - Little Girl
2. Toto - Little black dog
3. Scarecrow - Straw Man
4. Cowardly Lion - Cowardly Beast
5. Queen of the Mice - Mice Ruler
6. Toto - Little black dog
7. Wildcat - Wild Beast
8. Mice - Small creatures
9. Tin Man - Tin Creature
10. Lion - Ferocious Beast
11. Scarecrow - Straw Man
12. Wicked Witch of the East - Wicked Witch
13. Wizard - Magician
14. Green Girl - Emerald Girl
15. Soldier - Warrior

Please note that the nicknames provided are not official names, but rather informal or colloquial terms used to refer to the characters. The original names are provided in the text as well.
Question : 
Request 1. Please list up the nicknames of each characters in the list below refered in text.
Caution: Do not consider any word with 's as a name of the character, such as, Cowardly Lion's courage, Scarecrow's head, etc.
Dorothy, Tin Woodman, Scarecrow, Cowardly Lion, Queen of the Mice, Toto, Wildcat, Mice, Tin Man, Lion, Scarecrow, Wicked Witch of the Ea

In [40]:
import re

pattern = r'“[^”]*”|[^“”]+'
matches = re.findall(pattern, sample, flags=re.DOTALL)
cleaned_matchs = [match.strip() for match in matches if match.strip()]
len(cleaned_matchs)

1868

In [41]:
cleaned_matchs[:100]

['\ufeffThe Project Gutenberg eBook of The Wonderful Wizard of Oz\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: The Wonderful Wizard of Oz\n\n\nAuthor: L. Frank Baum\n\nRelease date: February 1, 1993 [eBook #55]\n                Most recently updated: March 30, 2021\n\nLanguage: English\n\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***\n[Illustration]\n\n\n\n\nThe Wonderful Wizard of Oz\n\nby L. Frank Baum\n\n\nThis book is dedicated to my good friend & comrade\nMy Wife\nL.F.B.\n\n\nContents\n\n Introduction\n Chapter I. The Cyclone\n Chapter II

In [42]:
!pip install -qqq nltk spacy

In [43]:
!pip show spacy

Name: spacy
Version: 3.7.2
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi, weasel
Required-by: en-core-web-sm, fastai


### Comparing the quality of Stemming and Lemmatization

#### Stemming

In [44]:
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
ls = LancasterStemmer()
ss = SnowballStemmer('english')

In [45]:
target_sentence='“What is it?”'
pattern='[^a-zA-Z0-9+]'
target_sentence=re.sub(pattern, ' ', target_sentence)
print(' '.join([ps.stem(i) for i in target_sentence.split()]))
print(' '.join([ls.stem(i) for i in target_sentence.split()]))
print(' '.join([ss.stem(i) for i in target_sentence.split()]))

what is it
what is it
what is it


#### Lematization

In [46]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser'])

In [47]:
doc = nlp(target_sentence)
' '.join([token.lemma_ for token in doc])

'  what be it  '

### Labeling each sentence.

In [48]:
!pip install -qqq torch transformers[torch] datasets faiss-gpu attrdict rank-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [51]:
from tqdm.auto import tqdm

In [72]:
# from collections import defaultdict
def labeling_sentence(list_of_sentences, characters):
    speech_verbs = ['say', 'speak', 'talk',
                    'reply', 'answer', 'add',
                    'continue', 'go on', 'cry',
                    'sigh', 'think', 'count',
                    'mumble', 'murmur', 'mention']
    final_list_of_sentences = []
    idx_character_spoke = {}
    for idx, sentence in tqdm(enumerate(list_of_sentences)):
        if sentence.startswith('"') and sentence.endswith('"'):
            final_list_of_sentences.append(f"Narrative: {sentence}")
        else: # when the sentence is a conversational speech
            try:
                prev_sentence = list_of_sentences[idx-1] if idx > 0 else '_'
                next_sentence = list_of_sentences[idx+1] if idx + 1 < len(list_of_sentences) else '_'
                if prev_sentence.strip()[0] != '"' and next_sentence.startswith('"') :
                    # if the previous line is narrative and next line is not narrative.
                    possible_speakers = []
                    for verb in speech_verbs:
                        # checking if the previous line contains speech verbs
                        doc = nlp(prev_sentence)
                        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
                        if verb in lemmatized_sentence:
                            for character in characters:
                                # checking if the previous line contains both sppech verbs and character
                                if character in prev_sentence:
                                    possible_speakers.append(character)
                    if len(possible_speakers) == 1:
                        # if there is only one possible speaker.
                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                        idx_character_spoke[idx] = possible_speakers[0]
                    elif len(possible_speakers) == 0:
                        continue # continue to search the sentence after the current sentence.
                    else: # if there are more than 1 candidate
                        characters_in_current_sentence = []
                        for character in characters:
                            if character in sentence:
                                characters_in_current_sentence.append(character)
                        if len(characters_in_current_sentence) == 1:
                            pattern = '[^a-zA-Z0-9+]'
                            if 'I' not in re.sub(pattern, ' ', sentence).split(): # e.g. "I, the wizard of Oz will kill you"
                                # if only the counterpart character is mentioned in the current sentence
                                if characters_in_current_sentence[0] in possible_speakers:
                                    possible_speakers.remove(characters_in_current_sentence[0])
                            else:
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]
                                continue
                        elif len(characters_in_current_sentence) > 1:
                            # if more than one character was mentioned in the current sentence.
                            # eg. "I am sure that Dorothy loves Toto"
                            # eg. "I am sure that I love her"
                            # eg. "The wizard of Oz will kill Dorothy"
                            # eg. "I, the wizard of Oz, will kill Dorothy"
                            # eg. "I thewizard of Oz, will kill Dorothy"
                            # eg. "I am the wizard of Oz and I will kill Dorothy"
                            temp_idx = {}
                            pattern = '[^a-zA-Z0-9+]'
                            temp_sent = re.sub(pattern, ' ', sentence)
                            if 'I' in temp_sent.split():
                                if temp_sent.split().count('I') == 1:
                                    for character in characters_in_current_sentence:
                                        temp_char = character.replace(' ', '')
                                        temp_sent = temp_sent.replace(character, temp_char)
                                        temp_idx[character] = abs(temp_sent.index('I') - temp_sent.index(temp_char))
                                    final_list_of_sentences.append(f"{min(temp_idx)}: {sentence}")
                                    idx_character_spoke[idx] = min(temp_idx)
                                    continue # continue to the next sentence.

                                elif temp_sent.split().count('I') > 1:
                                    t_char = []
                                    for character in characters_in_current_sentence:
                                        temp_char = character.replace(' ', '')
                                        t_char.append(temp_char)
                                        temp_sent = temp_sent.replace(character, temp_char)

                                    temp_sent = temp_sent.split()
                                    a = []
                                    for i, c in enumerate(temp_sent):
                                        if c == 'I':
                                            a.append(i)
                                    t_l = []
                                    for temp_char in t_char:
                                        for i in a:
                                            t_l.append(abs(i - temp_sent.index(temp_char)))
                                    temp_idx[character] = min(t_l)
                                    final_list_of_sentences.append(f"{min(temp_idx)}: {sentence}")
                                    idx_character_spoke[idx] = min(temp_idx)
                                    continue # continue to the next sentence.
                            else: # if there is no "I" in current_sentence
                                if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                                    if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                        # conversational
                                        if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                            final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                            idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                        else:
                                            if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                            idx_character_spoke[idx] = possible_speakers[0]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]
                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                        else:
                            if len(list(idx_character_spoke.keys())) > 2:
                                if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                                    if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                        # conversational
                                        if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                            final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                            idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                        else:
                                            if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                            idx_character_spoke[idx] = possible_speakers[0]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]
                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            elif len(list(idx_character_spoke.keys())) == 1:
                                if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                    # conversational
                                    if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                        final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                        idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]

                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            else:
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]

                elif prev_sentence.strip()[0] == '"' and next_sentence.strip()[0] != '"' :
                    # if the previous line is narrative and next line is not narrative.
                    possible_speakers = []
                    for verb in speech_verbs:
                        # checking if the previous line contains speech verbs
                        doc = nlp(next_sentence)
                        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
                        if verb in lemmatized_sentence:
                            for character in characters:
                                # checking if the previous line contains both sppech verbs and character
                                if character in next_sentence:
                                    possible_speakers.append(character)
                    if len(possible_speakers) == 1:
                        # if there is only one possible speaker.
                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                        idx_character_spoke[idx] = possible_speakers[0]
                    elif len(possible_speakers) == 0:
                        continue # continue to search the sentence after the current sentence.
                    else: # if there are more than 1 candidate
                        characters_in_current_sentence = []
                        for character in characters:
                            if character in sentence:
                                characters_in_current_sentence.append(character)
                        if len(characters_in_current_sentence) == 1:
                            pattern = '[^a-zA-Z0-9+]'
                            if 'I' not in re.sub(pattern, ' ', sentence).split(): # e.g. "I, the wizard of Oz will kill you"
                                # if only the counterpart character is mentioned in the current sentence
                                if characters_in_current_sentence[0] in possible_speakers:
                                    possible_speakers.remove(characters_in_current_sentence[0])

                            else:
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]
                                continue
                        elif len(characters_in_current_sentence) > 1:
                            # if more than one character was mentioned in the current sentence.
                            # eg. "I am sure that Dorothy loves Toto"
                            # eg. "I am sure that I love her"
                            # eg. "The wizard of Oz will kill Dorothy"
                            # eg. "I, the wizard of Oz, will kill Dorothy"
                            # eg. "I thewizard of Oz, will kill Dorothy"
                            # eg. "I am the wizard of Oz and I will kill Dorothy"
                            temp_idx = {}
                            pattern = '[^a-zA-Z0-9+]'
                            temp_sent = re.sub(pattern, ' ', sentence)
                            if 'I' in temp_sent.split():
                                if temp_sent.split().count('I') == 1:
                                    for character in characters_in_current_sentence:
                                        temp_char = character.replace(' ', '')
                                        temp_sent = temp_sent.replace(character, temp_char)
                                        temp_idx[character] = abs(temp_sent.index('I') - temp_sent.index(temp_char))
                                    final_list_of_sentences.append(f"{min(temp_idx)}: {sentence}")
                                    idx_character_spoke[idx] = min(temp_idx)
                                    continue # continue to the next sentence.

                                elif temp_sent.split().count('I') > 1:
                                    t_char = []
                                    for character in characters_in_current_sentence:
                                        temp_char = character.replace(' ', '')
                                        t_char.append(temp_char)
                                        temp_sent = temp_sent.replace(character, temp_char)

                                    temp_sent = temp_sent.split()
                                    a = []
                                    for i, c in enumerate(temp_sent):
                                        if c == 'I':
                                            a.append(i)
                                    t_l = []
                                    for temp_char in t_char:
                                        for i in a:
                                            t_l.append(abs(i - temp_sent.index(temp_char)))
                                    temp_idx[character] = min(t_l)
                                    final_list_of_sentences.append(f"{min(temp_idx)}: {sentence}")
                                    idx_character_spoke[idx] = min(temp_idx)
                                    continue # continue to the next sentence.
                            else: # if there is no "I" in current_sentence
                                if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                                    if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                        # conversational
                                        if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                            final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                            idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                        else:
                                            if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                            idx_character_spoke[idx] = possible_speakers[0]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]
                                else:
                                      if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                          possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                      final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                      idx_character_spoke[idx] = possible_speakers[0]
                        else:
                            if len(list(idx_character_spoke.keys())) > 2:
                                if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                                    if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                        # conversational
                                        if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                            final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                            idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                        else:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                            idx_character_spoke[idx] = possible_speakers[0]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]
                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            elif len(list(idx_character_spoke.keys())) == 1:
                                if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                    # conversational
                                    if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                        final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                        idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]

                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            else:
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]

                elif prev_sentence.strip()[0] != '"' and next_sentence.strip()[0] != '"' :
                    # if the previous line is narrative and next line is not narrative.
                    possible_speakers = []
                    for verb in speech_verbs:
                        # checking if the previous line contains speech verbs
                        doc = nlp(next_sentence)
                        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
                        if verb in lemmatized_sentence:
                            for character in characters:
                                # checking if the previous line contains both sppech verbs and character
                                if character in next_sentence:
                                    possible_speakers.append(character)
                        doc = nlp(prev_sentence)
                        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
                        if verb in lemmatized_sentence:
                            for character in characters:
                                # checking if the previous line contains both sppech verbs and character
                                if character in prev_sentence:
                                    possible_speakers.append(character)

                    if len(possible_speakers) == 1:
                        # if there is only one possible speaker.
                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                        idx_character_spoke[idx] = possible_speakers[0]
                    elif len(possible_speakers) == 0:
                        continue # continue to search the sentence after the current sentence.
                    else: # if there are more than 1 candidate
                        characters_in_current_sentence = []
                        for character in characters:
                            if character in sentence:
                                characters_in_current_sentence.append(character)
                        if len(characters_in_current_sentence) == 1:
                            pattern = '[^a-zA-Z0-9+]'
                            if 'I' not in re.sub(pattern, ' ', sentence).split(): # e.g. "I, the wizard of Oz will kill you"
                                # if only the counterpart character is mentioned in the current sentence
                                if characters_in_current_sentence[0] in possible_speakers:
                                    possible_speakers.remove(characters_in_current_sentence[0])
                            else:
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]
                                continue
                        elif len(characters_in_current_sentence) > 1:
                            # if more than one character was mentioned in the current sentence.
                            # eg. "I am sure that Dorothy loves Toto"
                            # eg. "I am sure that I love her"
                            # eg. "The wizard of Oz will kill Dorothy"
                            # eg. "I, the wizard of Oz, will kill Dorothy"
                            # eg. "I thewizard of Oz, will kill Dorothy"
                            # eg. "I am the wizard of Oz and I will kill Dorothy"
                            temp_idx = {}
                            pattern = '[^a-zA-Z0-9+]'
                            temp_sent = re.sub(pattern, ' ', sentence)
                            if 'I' in temp_sent.split():
                                if temp_sent.split().count('I') == 1:
                                    for character in characters_in_current_sentence:
                                        temp_char = character.replace(' ', '')
                                        temp_sent = temp_sent.replace(character, temp_char)
                                        temp_idx[character] = abs(temp_sent.index('I') - temp_sent.index(temp_char))
                                    final_list_of_sentences.append(f"{min(temp_idx)}: {sentence}")
                                    idx_character_spoke[idx] = min(temp_idx)
                                    continue # continue to the next sentence.

                                elif temp_sent.split().count('I') > 1:
                                    t_char = []
                                    for character in characters_in_current_sentence:
                                        temp_char = character.replace(' ', '')
                                        t_char.append(temp_char)
                                        temp_sent = temp_sent.replace(character, temp_char)

                                    temp_sent = temp_sent.split()
                                    a = []
                                    for i, c in enumerate(temp_sent):
                                        if c == 'I':
                                            a.append(i)
                                    t_l = []
                                    for temp_char in t_char:
                                        for i in a:
                                            t_l.append(abs(i - temp_sent.index(temp_char)))
                                    temp_idx[character] = min(t_l)
                                    final_list_of_sentences.append(f"{min(temp_idx)}: {sentence}")
                                    idx_character_spoke[idx] = min(temp_idx)
                                    continue # continue to the next sentence.
                            else: # if there is no "I" in current_sentence
                                if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                                    if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                        # conversational
                                        if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                            final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                            idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                        else:
                                            if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                            idx_character_spoke[idx] = possible_speakers[0]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]
                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                        else:
                            if len(list(idx_character_spoke.keys())) > 2:
                                if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                                    if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                        # conversational
                                        if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                            final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                            idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                        else:
                                            if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                            idx_character_spoke[idx] = possible_speakers[0]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]
                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            elif len(list(idx_character_spoke.keys())) == 1:
                                if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                    # conversational
                                    if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                        final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                        idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                    else:
                                        if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                            possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                        idx_character_spoke[idx] = possible_speakers[0]

                                else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            else:
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]

                else:
                    possible_speakers = []
                    for verb in speech_verbs:
                        # checking if the previous line contains speech verbs
                        doc = nlp(next_sentence)
                        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
                        if verb in lemmatized_sentence:
                            for character in characters:
                                # checking if the previous line contains both sppech verbs and character
                                if character in next_sentence:
                                    possible_speakers.append(character)
                        doc = nlp(prev_sentence)
                        lemmatized_sentence = ' '.join([token.lemma_ for token in doc])
                        if verb in lemmatized_sentence:
                            for character in characters:
                                # checking if the previous line contains both sppech verbs and character
                                if character in prev_sentence:
                                    possible_speakers.append(character)
                    if len(list(idx_character_spoke.keys())) > 2:
                        if list(idx_character_spoke.keys())[-2] in range(idx-4,idx-1):
                            if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                                # conversational
                                if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                    final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                    idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                                else:
                                    if list(idx_character_spoke.keys())[0] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                            else:
                                    if list(idx_character_spoke.keys())[-1] in possible_speakers:
                                        possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                    final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                    idx_character_spoke[idx] = possible_speakers[0]
                        else:
                            if list(idx_character_spoke.keys())[0] in possible_speakers:
                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                            idx_character_spoke[idx] = possible_speakers[0]
                    elif len(list(idx_character_spoke.keys())) == 1:
                        if list(idx_character_spoke.keys())[-1] in range(idx-2,idx):
                            # conversational
                            if len(list(set(list(idx_character_spoke.keys())[-2:]))) == 2:
                                final_list_of_sentences.append(f"{list(idx_character_spoke.keys())[-2]}: {sentence}")
                                idx_character_spoke[idx] = list(idx_character_spoke.keys())[-2]
                            else:
                                if list(idx_character_spoke.keys())[0] in possible_speakers:
                                    possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                                final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                                idx_character_spoke[idx] = possible_speakers[0]

                        else:
                            if list(idx_character_spoke.keys())[0] in possible_speakers:
                                possible_speakers.remove(list(idx_character_spoke.keys())[-1])
                            final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                            idx_character_spoke[idx] = possible_speakers[0]
                    else:
                        final_list_of_sentences.append(f"{possible_speakers[0]}: {sentence}")
                        idx_character_spoke[idx] = possible_speakers[0]
            except:
                final_list_of_sentences.append(f"Unidentified: {sentence}")

    return final_list_of_sentences

In [ ]:
labeled_text = labeling_sentence(cleaned_matchs, characters)
labeled_text[100:200]

0it [00:00, ?it/s]

In [7]:
if 1 in range(0, 5):
    print('True')

True


In [ ]:
def attribute_dialogue(list_of_sentences, characters):
    attributed_dialogues = []
    extra_character_count = 1

    for idx, text in tqdm(enumerate(list_of_sentences)):
        if text.startswith('"') and text.endswith('"'):
            prev_text = list_of_sentences[idx-1] if idx > 0 else ""
            next_text = list_of_sentences[idx+1] if idx+1 < len(list_of_sentences) else ""

            speaker = None
            for character in characters:
                if character in prev_text or character in next_text:
                    speaker = character

            if not speaker:
                speaker = f'Extra {extra_character_count}'
                extra_character_count += 1
            attributed_dialogues.append((speaker, text))
        else:
            attributed_dialogues.append(('Narrator', text))
    return attributed_dialogues


In [32]:
from tqdm.auto import tqdm

full_text_splitted = ''
for text in cleaned_matchs:
    full_text_splitted += f'{text}\n'

In [33]:
print(full_text_splitted)

﻿The Project Gutenberg eBook of The Wonderful Wizard of Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Wonderful Wizard of Oz


Author: L. Frank Baum

Release date: February 1, 1993 [eBook #55]
                Most recently updated: March 30, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***
[Illustration]




The Wonderful Wizard of Oz

by L. Frank Baum


This book is dedicated to my good friend & comrade
My Wife
L.F.B.


Contents

 Introduction
 Chapter I. The Cyclone
 Chapter II. The Council with the Munchkins
 Chapter III. Ho

In [26]:
from google.colab import runtime
runtime.unassign()